# Let's call all libraries and define city and road variables

In [1]:
import networkx as nx

# Write all city names as variables for simplicity (to be able to call faster with no typos)
ATLANTA = "Atlanta"
BOSTON = "Boston"
CALGARY = "Calgary"
CHARLESTON = "Charleston"
CHICAGO = "Chicago"
DALLAS = "Dallas"
DENVER = "Denver"
DULUTH = "Duluth"
EL_PASO = "El Paso"
HELENA = "Helena"
HOUSTON = "Houston"
KANSAS_CITY = "Kansas City"
LAS_VEGAS = "Las Vegas"
LITTLE_ROCK = "Little Rock"
LOS_ANGELES = "Los Angeles"
MIAMI = "Miami"
MONTREAL = "Montreal"
NASHVILLE = "Nashville"
NEW_ORLEANS = "New Orleans"
NEW_YORK = "New York"
OKLAHOMA_CITY = "Oklahoma City"
OMAHA = "Omaha"
PHOENIX = "Phoenix"
PITTSBURGH = "Pittsburgh"
PORTLAND = "Portland"
RALEIGH = "Raleigh"
SAINT_LOUIS = "Saint Louis"
SALT_LAKE_CITY = "Salt Lake City"
SAN_FRANCISCO = "San Francisco"
SANTA_FE = "Santa Fe"
SAULT_ST_MARIE = "Sault St. Marie"
SEATTLE = "Seattle"
TORONTO = "Toronto"
VANCOUVER = "Vancouver"
WASHINGTON = "Washington"
WINNIPEG = "Winnipeg"

# VERTICES are list of all cities
with open('cities.txt') as f:
    VERTICES = [i.rstrip() for i in f] # stripping "\n" from end of lines and seperating all lines
    
# EDGES are roads connecting 2 cities (City A,City B,Distance,Color)
with open('roads.txt') as f:
    next(f) # skipping header
    EDGES = [tuple(i.strip().split(',')) for i in f] # stripping "\n" and making other , seperated terms a tuple

# TICKETS are given by the game. The goal is to connect those 2 cities, 3rd input is the point you get from completing
# (City A,City B,Points)
with open('tickets.txt') as f:
    next(f) # skipping header
    TICKETS = [tuple(i.strip().split(',')) for i in f] # stripping "\n" and making other , seperated terms a tuple

# Build bidirectional weighted graph from the VERTICES and EDGES

In [2]:
G = nx.MultiGraph()
G.add_nodes_from(VERTICES) #add vertices

for edge in EDGES:
    G.add_edge(edge[0], edge[1], weight = int(edge[2]), colour = edge[3])
print(G["Montreal"])
print(G["Kansas City"])

{'Sault St. Marie': {0: {'weight': 5, 'colour': 'K'}}, 'Toronto': {0: {'weight': 3, 'colour': 'X'}}, 'New York': {0: {'weight': 3, 'colour': 'B'}}, 'Boston': {0: {'weight': 2, 'colour': 'X'}, 1: {'weight': 2, 'colour': 'X'}}}
{'Omaha': {0: {'weight': 1, 'colour': 'X'}, 1: {'weight': 1, 'colour': 'X'}}, 'Saint Louis': {0: {'weight': 2, 'colour': 'B'}, 1: {'weight': 2, 'colour': 'P'}}, 'Oklahoma City': {0: {'weight': 2, 'colour': 'X'}, 1: {'weight': 2, 'colour': 'X'}}, 'Denver': {0: {'weight': 4, 'colour': 'K'}, 1: {'weight': 4, 'colour': 'O'}}}


In [3]:
print(nx.dijkstra_path(G,SEATTLE, NASHVILLE))
print(nx.dijkstra_path_length(G,SEATTLE, NASHVILLE))

['Seattle', 'Helena', 'Omaha', 'Kansas City', 'Saint Louis', 'Nashville']
16


# Steiner tree tries to find minimum sum of weights to connect given nods

In [5]:
# Note: Finding optimal Steiner Tree is an NP problem and the function nx.steiner_tree() just approximates 
#the best solution
# Had "no attribute" problems for this function. It is copied from the documentation, made some slight changes and 
# removed unnecessary functions:

from networkx.utils import pairwise 

def steiner_tree(G, terminal_nodes, weight="weight", method=None):
    if method is None:
        method = "mehlhorn"
        
    try:
        algo = _mehlhorn_steiner_tree
    except KeyError as e:
        raise ValueError(f"{method} is not a valid choice for an algorithm.") from e

    edges = algo(G, terminal_nodes, weight)
    # For multigraph we should add the minimal weight edge keys
    if G.is_multigraph():
        edges = (
            (u, v, min(G[u][v], key=lambda k: G[u][v][k][weight])) for u, v in edges
        )
    T = G.edge_subgraph(edges)
    return T

def _mehlhorn_steiner_tree(G, terminal_nodes, weight):
    paths = nx.multi_source_dijkstra_path(G, terminal_nodes)

    d_1 = {}
    s = {}
    for v in G.nodes():
        s[v] = paths[v][0]
        d_1[(v, s[v])] = len(paths[v]) - 1

    # G1-G4 names match those from the Mehlhorn 1988 paper.
    G_1_prime = nx.Graph()
    for u, v, data in G.edges(data=True):
        su, sv = s[u], s[v]
        weight_here = d_1[(u, su)] + data.get(weight, 1) + d_1[(v, sv)]
        if not G_1_prime.has_edge(su, sv):
            G_1_prime.add_edge(su, sv, weight=weight_here)
        else:
            new_weight = min(weight_here, G_1_prime[su][sv]["weight"])
            G_1_prime.add_edge(su, sv, weight=new_weight)

    G_2 = nx.minimum_spanning_edges(G_1_prime, data=True)

    G_3 = nx.Graph()
    for u, v, d in G_2:
        path = nx.shortest_path(G, u, v, weight)
        for n1, n2 in pairwise(path):
            G_3.add_edge(n1, n2)

    G_3_mst = list(nx.minimum_spanning_edges(G_3, data=False))
    if G.is_multigraph():
        G_3_mst = (
            (u, v, min(G[u][v], key=lambda k: G[u][v][k][weight])) for u, v in G_3_mst
        )
    G_4 = G.edge_subgraph(G_3_mst).copy()
    _remove_nonterminal_leaves(G_4, terminal_nodes)
    return G_4.edges()

def _remove_nonterminal_leaves(G, terminals):
    terminals_set = set(terminals)
    for n in list(G.nodes):
        if n not in terminals_set and G.degree(n) == 1:
            G.remove_node(n)

In [16]:
G2 =steiner_tree(G, [ATLANTA, NEW_ORLEANS, OMAHA, SEATTLE])
print(list(G2)) # what it displays is the minimal list of nodes we must use to visit ATLANTA, NEW_ORLEANS, OMAHA, SEATTLE

# Or to get the necessary edges we can do:
print("\n",_mehlhorn_steiner_tree(G, [ATLANTA, NEW_ORLEANS, OMAHA, SEATTLE], weight = "weight"))

['Seattle', 'Helena', 'Omaha', 'Nashville', 'New Orleans', 'Saint Louis', 'Kansas City', 'Atlanta']

 [('Seattle', 'Helena'), ('Helena', 'Omaha'), ('Omaha', 'Kansas City'), ('Nashville', 'Saint Louis'), ('Nashville', 'Atlanta'), ('Saint Louis', 'Kansas City'), ('New Orleans', 'Atlanta')]
